In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from PIL import Image
import time
import yaml
import numpy as np
from tqdm import tqdm
from torchvision import transforms
from torchvision.transforms import ToPILImage, ToTensor
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim import Adam
from torchvision.utils import save_image
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
sys.path.append("models")

import models
import utils
from utils import make_coord,set_save_path,ssim
import datasets
from test import eval_psnr,batched_predict

from models import losses
from models.liif import LIIF
from models.discriminator import Discriminator
from models.losses import AdversarialLoss
%matplotlib inline

In [20]:
img = Image.open(r'load\div2k\DIV2K_valid_LR_bicubic\X2\0802x2.png')
#img = transforms.Resize((int(img.height/2),int(img.width/2)),Image.BICUBIC)(img)
timg = transforms.ToTensor()(img) #[3,LR_H,LR_W]
edgemap = edge.laplacian_kernel(timg.unsqueeze(0).cuda())
transforms.ToPILImage()(edgemap[0]).save('0802x2edge.jpg')

In [20]:
#pytorch 求LPIPS
 
import torch
import lpips
import os
 
use_gpu = False         # Whether to use GPU
spatial = False         # Return a spatial map of perceptual distance.
 
# Linearly calibrated models (LPIPS)
loss_fn = lpips.LPIPS(net='vgg', spatial=spatial) # Can also set net = 'squeeze' or 'vgg'
# loss_fn = lpips.LPIPS(net='alex', spatial=spatial, lpips=False) # Can also set net = 'squeeze' or 'vgg'
 
if(use_gpu):
	loss_fn.cuda()
	

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: d:\Miniconda3\envs\pytorch\lib\site-packages\lpips\weights\v0.1\vgg.pth


In [19]:
loss_fn

LPIPS(
  (scaling_layer): ScalingLayer()
  (net): vgg16(
    (slice1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
    (slice2): Sequential(
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
    )
    (slice3): Sequential(
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, 

In [23]:
## Example usage with dummy tensors
rood_path = r'D:\Project\results\faces'
hr_path = r'E:\Code\Python\datas\selfWHURS\WHURS19-test\GT'
sr_path = r'E:\Code\Python\liif-self\result\WHURS19_edsrblx4'

hr_path_list = []
sr_path_list = []
## path in net is already exist

for root, _, fnames in sorted(os.walk(hr_path, followlinks=True)):
	for fname in fnames:
		path = os.path.join(hr_path, fname)
		hr_path_list.append(path)

for root, _, fnames in sorted(os.walk(sr_path, followlinks=True)):
	for fname in fnames:
		path = os.path.join(sr_path, fname)
		sr_path_list.append(path)


In [26]:
dist_ = []
for i in range(len(hr_path_list)):
	hr_img = lpips.im2tensor(lpips.load_image(hr_path_list[i]))
	sr_img = lpips.im2tensor(lpips.load_image(sr_path_list[i]))
	if(use_gpu):
		hr_img = hr_img.cuda()
		sr_img = sr_img.cuda()
	dist = loss_fn.forward(hr_img, sr_img)
	dist_.append(dist.mean().item())
print('Avarage Distances: %.3f' % (sum(dist_)/len(hr_path_list)))

Avarage Distances: 0.304


In [21]:
hr_img = lpips.im2tensor(lpips.load_image(r'E:\Code\Python\liif-self\result\WHURS19_edsrblx4\airport_41.png'))
sr_img = lpips.im2tensor(lpips.load_image(r'E:\Code\Python\datas\selfWHURS\WHURS19-test\GT\airport_41.jpg'))
if(use_gpu):
    hr_img = hr_img.cuda()
    sr_img = sr_img.cuda()
dist = loss_fn.forward(hr_img, sr_img)
dist.mean()

tensor(0.2441, grad_fn=<MeanBackward0>)

In [22]:
hr_img = lpips.im2tensor(lpips.load_image(r'E:\Code\Python\liif-self\result\WHURS19_samx_L0Sgradx4\airport_41.png'))
sr_img = lpips.im2tensor(lpips.load_image(r'E:\Code\Python\datas\selfWHURS\WHURS19-test\GT\airport_41.jpg'))
if(use_gpu):
    hr_img = hr_img.cuda()
    sr_img = sr_img.cuda()
dist = loss_fn.forward(hr_img, sr_img)
dist.mean()

tensor(0.2435, grad_fn=<MeanBackward0>)